特征提取文件 生成feature_answer_base2.h5

In [1]:
import pandas as pd
import time

time_start=time.time()




In [2]:
import numpy as np
from tqdm import tqdm
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in tqdm(df.columns):
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])

In [4]:
invite_info=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
invite_info_evaluate=pd.read_csv('invite_info_evaluate_2_0926.txt',sep='\s+',names=['qid','uid','time'])

In [5]:
from tqdm import tqdm
tqdm.pandas()

In [6]:
answer_info=answer_info[['uid','qid','atime','picture','video','numword','thumbsup',
                  'thumbsdown','comment','collect','thank','report','nohelp','oppose']]
#,'recommend','fine'

In [7]:
answer_info['atime']=answer_info['atime'].progress_apply(lambda x:int(x[1:].split('-H')[0]))

100%|██████████| 4513735/4513735 [00:09<00:00, 461460.99it/s]


In [8]:
all_data=pd.concat([invite_info[['qid','uid','time']],invite_info_evaluate])
all_data['time']=all_data['time'].progress_apply(lambda x:int(x[1:].split('-H')[0]))

100%|██████████| 10630880/10630880 [00:19<00:00, 543863.50it/s]


In [9]:
all_data=pd.merge(all_data,answer_info,how='inner',on='uid')

In [10]:
all_data=all_data[all_data['qid_x']!=all_data['qid_y']]


In [11]:
all_data['sub_time']=all_data['time']-all_data['atime']
all_data=all_data[all_data['sub_time']>0]

In [12]:
all_data.drop(['qid_y','time','atime'],axis=1,inplace=True)

In [13]:
all_data=all_data.groupby(['uid','qid_x']).agg(['mean','max','sum','min','median','std'])

In [15]:
all_data=all_data.reset_index()

In [16]:
all_data.head()

uid        qid_x picture                        video      ...  \
                               mean max sum min median std  mean max  ...   
0  M1000000382  Q1282394327     1.0   1   1   1    1.0 NaN   0.0   0  ...   
1  M1000000382  Q2418257178     1.0   1   1   1    1.0 NaN   0.0   0  ...   
2  M1000000382  Q2557202265     1.0   1   1   1    1.0 NaN   0.0   0  ...   
3  M1000000382  Q3739876694     1.0   1   1   1    1.0 NaN   0.0   0  ...   
4  M1000000382  Q3910864809     1.0   1   1   1    1.0 NaN   0.0   0  ...   

  oppose                sub_time                         
     sum min median std     mean max sum min median std  
0      0   0    0.0 NaN     17.0  17  17  17   17.0 NaN  
1      0   0    0.0 NaN     19.0  19  19  19   19.0 NaN  
2      0   0    0.0 NaN      1.0   1   1   1    1.0 NaN  
3      0   0    0.0 NaN     23.0  23  23  23   23.0 NaN  
4      0   0    0.0 NaN      3.0   3   3   3    3.0 NaN  

[5 rows x 74 columns]

In [17]:
col=[]
for i in ['picture','video','numword','thumbsup',
                  'thumbsdown','comment','collect','thank','report','nohelp','oppose','sub_time']:
    col.append(i+'_mean')
    col.append(i+'_max')
    col.append(i+'_sum')
    col.append(i+'_min')
    col.append(i+'_median')
    col.append(i+'std')

In [18]:
all_data.columns=['uid','qid']+col

In [19]:
len(all_data)

7459968

In [20]:
all_data.head()

,uid,qid,picture_mean,picture_max,picture_sum,picture_min,picture_median,picturestd,video_mean,video_max,...,oppose_sum,oppose_min,oppose_median,opposestd,sub_time_mean,sub_time_max,sub_time_sum,sub_time_min,sub_time_median,sub_timestd
0,M1000000382,Q1282394327,1.0,1,1,1,1.0,NaN,0.0,0,...,0,0,0.0,NaN,17.0,17,17,17,17.0,NaN
1,M1000000382,Q2418257178,1.0,1,1,1,1.0,NaN,0.0,0,...,0,0,0.0,NaN,19.0,19,19,19,19.0,NaN
2,M1000000382,Q2557202265,1.0,1,1,1,1.0,NaN,0.0,0,...,0,0,0.0,NaN,1.0,1,1,1,1.0,NaN
3,M1000000382,Q3739876694,1.0,1,1,1,1.0,NaN,0.0,0,...,0,0,0.0,NaN,23.0,23,23,23,23.0,NaN
4,M1000000382,Q3910864809,1.0,1,1,1,1.0,NaN,0.0,0,...,0,0,0.0,NaN,3.0,3,3,3,3.0,NaN


In [21]:
all_data=reduce_mem_usage(all_data)

100%|██████████| 74/74 [00:14<00:00,  5.76it/s]

Mem. usage decreased to 1287.70 Mb (69.4% reduction)


In [22]:
all_data.to_hdf('feature_answer_base2.h5', key='data',index=False)

In [23]:
#针对qid统计